## Lesson 06: Intro to Simulation and Simple Linear Regression

In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

## Sampling from a Table

The `sample` function (from the `datascience` package) draws uniformly at random with replacement from a `Table` object.

In [ ]:
faces = np.arange(1, 7)
die = Table().with_columns('Face', faces)
type(die)

In [ ]:
die

In [ ]:
die.sample(7)

### Empirical Histograms

"Empirical distributions, on the other hand, are distributions of observed data. They can be visualized by empirical histograms." [[2]](https://inferentialthinking.com/chapters/10/1/Empirical_Distributions.html?highlight=simple%20random%20sample#id1)

In [ ]:
# Set the bin width to 1 centered on the integers 1 through 6
die_bins = np.arange(0.5, 6.6, 1) 

def empirical_hist_die(n):
    """Sample n times time with replacement from the die Table 
       and make a histogram of the distribution.
    """
    die.sample(n).hist(bins=die_bins)

In [ ]:
empirical_hist_die(100000)

**Question 1.** What do you notice as the sample size increases?

A `Table` can be made from an `array` object. 

- The `make_array()` function (from the `datascience` package) will make an array.

- The `.with_column()` function will make a table from an array.

In [ ]:
coin = Table().with_column('coin', make_array('H','T'))
type(coin)

In [ ]:
coin

In [ ]:
coin.sample(10)

### Coin Toss

Let's simulate the number of heads in 100 coin tosses.

In [ ]:
sum(coin.sample(100).column(0) == 'H')

In [ ]:
We can write a function to simulate this experiment one time.

In [ ]:
def num_heads():
    """Simulate the number of heads in 100 coin tosses.
    """
    return sum(coin.sample(100).column(0) == 'H')

Now let's simulate this experiment 1000 times by using our `num_heads` function in a `for` loop.

In [ ]:
# Set the number of times to repeat the experiment
repetitions = 1000

# Initialize an array to store the outcome from each experiment
outcomes = make_array()

# for loop to repeat the experiment and store the outcome to an array
for i in np.arange(repetitions):
    outcomes = np.append(outcomes, num_heads())

Finally we can visualize the distribution of the outcomes by making a histogram by creating a `Table` from the `outcomes` array and plotting a histogram using the `.hist()` function.

In [ ]:
Table().with_column('Heads', outcomes).hist()

**Question 2.** If someone made a claim that they flipped a coin 100 times and got 55 heads, would you think it happened by chance or due to their skill level with regards to being able to flip a coin? What if they got 65 heads?

## Sampling from a Population

Let's pretend that our data set represents every single flight delay for United Airlines (i.e., we'll think of it as the population).

In [ ]:
united = Table.read_table('data/united.csv')
united

**Question 3.** What do you notice about the delay times? Use the code cells below to explore the data.

In [ ]:
...

In [ ]:
...

In [ ]:
delay_bins = np.arange(-20, 201, 10)
united.hist('Delay', bins=delay_bins, unit='minute')

Now let's sample from it with replacement and make a histogram. What do you think will happen as the sample size increases?

In [ ]:
def empirical_hist_delay(n):
    """Sample n times time with replacement from the united 
       Table and make a histogram of the distribution.
    """
    united.sample(n).hist('Delay', bins=delay_bins, unit='minute')

In [ ]:
empirical_hist_delay(10000)

**Question 4.** What do you notice as the sample size increases?

## Simple Linear Regression

In the cell below there are some functions for plotting. You don't have to understand how any of the functions in the cell work, since they use things we have let to learn. 

In [ ]:
def resize_window(lim=3.5):
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    
def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='r'):
    y = x*slope+intercept
    plt.plot(x, y, color=color)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plt.plot(x, y, color=color)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x+(np.sqrt(1-r**2))*z
    return x, y

def r_table(r):
    """
    Generate a table of 1000 x,y data points in standard units
    whose correlation is approximately equal to r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

## Prediction lines

In [ ]:
example = r_table(0.99)
example.show(3)

In [ ]:
example.scatter('x', 'y')

In [ ]:
def nn_prediction_example(x_val, how_close=0.25):
    """Predicts y-value for x based on the example table, 
       using points within the specified number of standard units.
    """
    neighbors = example.where(
        'x', 
        are.between(x_val-how_close, x_val+how_close)
    )
    return np.mean(neighbors.column('y'))    

In [ ]:
nn_prediction_example(-0.5)

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
example.scatter('x')

In [ ]:
example.scatter('x')
draw_line(slope=1, color='red')
resize_window()

In [ ]:
example = r_table(0)
example.scatter('x', 'y')
draw_line(slope=0.5, color='red')
resize_window()

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
example = example.with_column(
    'Predicted y', example.apply(nn_prediction_example, 'x'))

Table.interactive_plots()
example.scatter('x')
draw_line(slope=0.5, color='red')
resize_window()

In [ ]:
example = r_table(0.5)
example.scatter('x', 'y')
resize_window()

In [ ]:
example = r_table(0.5)

example = example.with_column(
    'Predicted y', example.apply(nn_prediction_example, 'x'))
example.scatter('x', 'y')
draw_vertical_line(1.5, color='blue')
draw_line(slope=1, intercept=0, color='red')
resize_window()

In [ ]:
example = example.with_column('Predicted y', example.apply(nn_prediction_example, 'x'))
example.scatter('x')
draw_vertical_line(1.5, color='blue')
draw_line(slope=1, color='red')
resize_window()

In [ ]:
example.scatter('x')
draw_line(slope=1, intercept=0, color='red')
draw_line(slope=0.5, intercept=0, color='dodgerblue')
resize_window()

## Linear Regression: Defining the Line

In [ ]:
def standard_units(x):
    """Converts an array x to standard units.
    """
    return (x-np.mean(x))/np.std(x)

def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su*y_su)

**Question 5.** Finish the code to find the slope and intercept of the regression line.

**Hint:** See the lesson for how to define these.

In [ ]:
def slope(t, x, y):
    ...

def intercept(t, x, y):
    ...

In [ ]:
example = r_table(0.5)
slope(example, 'x', 'y')

## Galton height data

In [ ]:
galton = Table.read_table('data/galton.csv')

heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight'))
heights

In [ ]:
m = galton.where('gender', 'female')
np.mean(m.column('childHeight'))

In [ ]:
np.mean(m.column('mother'))

In [ ]:
def nn_prediction_galton(h):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of h.
    
    The prediction is the average height of the children 
    whose midparent height is in the range h plus or minus 0.5 inches.
    """
    neighbors = heights.where(
        'MidParent', are.between(h - 0.5, h + 0.5))
    return np.mean(neighbors.column('Child'))

In [ ]:
heights_with_predictions = heights.with_column(
    'Average neighbor prediction', 
    heights.apply(nn_prediction_galton, 'MidParent'))

**Question 6.** Make sure you define the functions slope and intercept before moving on.

In [ ]:
galton_slope = ...
galton_intercept = ...
galton_slope, galton_intercept

In [ ]:
...

Let's make a prediction for the height of these parents' children.


In [ ]:
heights_with_predictions.where('MidParent', are.equal_to(69.48))

In [ ]:
heights_with_predictions = heights_with_predictions.with_column(
    'Regression Prediction', 
    galton_slope*heights.column('MidParent') + galton_intercept
)
heights_with_predictions

In [ ]:
heights_with_predictions.scatter('MidParent')